In [11]:
## Imports
import tensorflow as tf
import pandas as pd
import os
import glob

In [12]:
def preprocess_image(file_path, label):
    image = tf.io.read_file(file_path)
    image = tf.image.decode_png(image, channels=3)  # 3 channels for pretrained models
    image = tf.image.resize(image, [224, 224])
    image = tf.cast(image, tf.float32) / 255.0
    return image, label


In [13]:
import pickle

with open('mura_train.pkl', 'rb') as f:
    train_images, train_labels = pickle.load(f)

with open('mura_val.pkl', 'rb') as f:
    val_images, val_labels = pickle.load(f)


In [14]:
train_ds = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
train_ds = train_ds.map(preprocess_image).shuffle(1000).batch(32).prefetch(tf.data.AUTOTUNE)

val_ds = tf.data.Dataset.from_tensor_slices((val_images, val_labels))
val_ds = val_ds.map(preprocess_image).batch(32).prefetch(tf.data.AUTOTUNE)


In [15]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras import Model, Input

In [16]:
# Load pretrained model without the top
base_model = ResNet50(include_top=False, weights='imagenet', input_shape=(224, 224, 3))


In [17]:
# Freeze base model
base_model.trainable = False

In [18]:
# Add new head
inputs = Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
outputs = Dense(1, activation='sigmoid')(x)

model = Model(inputs=inputs, outputs=outputs)

In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

history_1 = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    callbacks=[early_stopping]
)


Epoch 1/10
1151/1151 ━━━━━━━━━━━━━━━━━━━━ 4102s 4s/step - accuracy: 0.8059 - loss: 0.4446 - val_accuracy: 0.5221 - val_loss: 2.0166
Epoch 2/10
 580/1151 ━━━━━━━━━━━━━━━━━━━━ 31:06 3s/step - accuracy: 0.7845 - loss: 0.5914

In [ ]:
# Unfreeze all layers
base_model.trainable = True

# Recompile with lower LR
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss='binary_crossentropy',
              metrics=['accuracy'])

history_2 = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    callbacks=[early_stopping]
)


In [ ]:
def plot_history(history, title):
    plt.figure()
    plt.plot(history.history['accuracy'], label='train acc')
    plt.plot(history.history['val_accuracy'], label='val acc')
    plt.title(title)
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid()
    plt.show()

plot_history(history_1, "Transfer Learning (Frozen Base)")
plot_history(history_2, "Fine-Tuning All Layers")
